In [1]:
import pyspark
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkSession.builder.appName('PySparkShell').getOrCreate()
sc

In [2]:
df = sc.read.option('header','true').csv('gs://dsci-551/Data/patients_data.csv')

In [3]:
df.printSchema()

root
 |-- subject_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- dod: string (nullable = true)
 |-- dod_hosp: string (nullable = true)
 |-- dod_ssn: string (nullable = true)
 |-- expire_flag: string (nullable = true)



In [4]:
df = df.withColumn('dob', 
                   to_date(unix_timestamp(col('dob'), 'MM-dd-yyyy').cast("timestamp")))

In [5]:
df2 = sc.read.option('header','true').csv('gs://dsci-551/Data/icustays_data.csv')

In [6]:
df2 = df2.withColumn('intime', 
                   to_date(unix_timestamp(col('intime'), 'MM-dd-yyyy').cast("timestamp")))
df2 = df2.withColumn('outtime', 
                   to_date(unix_timestamp(col('outtime'), 'MM-dd-yyyy').cast("timestamp")))
# df2.show()
df2.printSchema()

root
 |-- subject_id: string (nullable = true)
 |-- hadm_id: string (nullable = true)
 |-- icu_stay_id: string (nullable = true)
 |-- dbsource: string (nullable = true)
 |-- first_careunit: string (nullable = true)
 |-- last_careunit: string (nullable = true)
 |-- firstwardid: string (nullable = true)
 |-- lastwardid: string (nullable = true)
 |-- intime: date (nullable = true)
 |-- outtime: date (nullable = true)



In [7]:
df3 = sc.read.option('header','true').csv('gs://dsci-551/Data/services_data.csv')

In [8]:
df4 = sc.read.option('header','true').csv('gs://dsci-551/Data/admissions_data.csv')

In [9]:
df5 = sc.read.option('header','true').csv('gs://dsci-551/Data/prescriptions_data.csv')

In [12]:
df.createOrReplaceTempView("patients_db")
search_1_count = sc.sql("SELECT count(*) as tot_count from patients_db where gender = 'M' and year(dob)<2000")
# search_1_count.show()

In [13]:
search_1 = sc.sql("SELECT * from patients_db where gender = 'M' and year(dob)<2000")
search_1.show(1203)

+----------+------+----------+----------+----------+----------+-----------+
|subject_id|gender|       dob|       dod|  dod_hosp|   dod_ssn|expire_flag|
+----------+------+----------+----------+----------+----------+-----------+
|         2|     M|1987-11-22|      None|      None|      None|          0|
|         4|     M|1972-09-14|06-03-2015|06-03-2015|      None|          1|
|         5|     M|1977-08-13|07-01-2020|07-01-2020|07-01-2020|          1|
|         6|     M|1993-06-17|10-28-2019|10-28-2019|10-28-2019|          1|
|         7|     M|1962-07-18|11-12-2012|11-12-2012|      None|          1|
|         8|     M|1969-07-10|      None|      None|      None|          0|
|         9|     M|1998-12-15|      None|      None|      None|          0|
|        13|     M|1954-05-24|      None|      None|      None|          0|
|        14|     M|1957-09-09|12-24-2015|12-24-2015|      None|          1|
|        15|     M|1995-02-23|      None|      None|      None|          0|
|        19|

In [15]:
df4.createOrReplaceTempView("admissions_db")
search_2_count = sc.sql("""SELECT count(*) as tot_count from admissions_db where religion = 'CATHOLIC' 
and admission_location = 'EMERGENCY ROOM ADMIT' and discharge_location = 'HOME'""")
# search_2_count.show()

In [16]:
search_2 = sc.sql("""SELECT * from admissions_db where religion = 'CATHOLIC' and 
admission_location = 'EMERGENCY ROOM ADMIT' and discharge_location = 'HOME'""")
search_2.show(101)

+----+----------+-------+----------+--------------+--------------+--------------------+------------------+----------+--------+-----------------+--------------------+--------------------+--------+
| _c0|subject_id|hadm_id|admit_date|discharge_date|admission_type|  admission_location|discharge_location| insurance|religion|   marital_status|           ethnicity|           diagnosis|language|
+----+----------+-------+----------+--------------+--------------+--------------------+------------------+----------+--------+-----------------+--------------------+--------------------+--------+
|  12|        13|   5827|09-05-2004|    03-21-2007|     EMERGENCY|EMERGENCY ROOM ADMIT|              HOME|  Medicare|CATHOLIC|          MARRIED|HISPANIC/LATINO -...|               FEVER|    SPAN|
|  19|        20|   5834|10-05-2010|    06-27-2011|     EMERGENCY|EMERGENCY ROOM ADMIT|              HOME|  Medicare|CATHOLIC|          WIDOWED|               WHITE|  UTI/PYELONEPHRITIS|    SPAN|
|  62|        63|   

In [19]:
df2.createOrReplaceTempView("icustays_db")
search_3_count = sc.sql("SELECT count(*) from icustays_db where year(outtime)-year(intime)>2 ")
# search_3_count.show()

In [18]:
search_3 = sc.sql("SELECT * from icustays_db where year(outtime)-year(intime)>2 ")
search_3.show(764)

+----------+-------+-----------+----------+--------------+-------------+-----------+----------+----------+----------+
|subject_id|hadm_id|icu_stay_id|  dbsource|first_careunit|last_careunit|firstwardid|lastwardid|    intime|   outtime|
+----------+-------+-----------+----------+--------------+-------------+-----------+----------+----------+----------+
|         1|   5815|       9417|metavision|           CCU|          CCU|         12|        12|2017-12-06|2020-07-07|
|         4|   5818|       9420|   carevue|         TSICU|        TSICU|         57|        57|1989-04-26|1992-08-24|
|         5|   5819|       9421|   carevue|           CCU|          CCU|         15|        15|1983-08-03|1986-07-16|
|        13|   5827|       9429|metavision|          MICU|         MICU|         57|        57|2004-09-05|2007-03-21|
|        15|   5829|       9431|metavision|           CCU|          CCU|          7|         7|2026-11-10|2029-12-20|
|        22|   5836|       9438|metavision|           CC

In [20]:
df3.createOrReplaceTempView("services_db")
search_4_count = sc.sql("SELECT count(*) as tot_count from services_db where curr_serv='SURG' ")
# search_4_count.show()

In [21]:
search_4 = sc.sql("SELECT * from services_db where curr_serv='SURG' ")
search_4.show(251)

+----+------+----------+-------+-------------------+---------+---------+
| _c0|row_id|subject_id|hadm_id|       transfertime|prev_serv|curr_serv|
+----+------+----------+-------+-------------------+---------+---------+
|  48|    49|        27|   5841|01-13-2011 00:00:00|     NONE|     SURG|
|  56|    57|        31|   5845|02-25-2010 00:00:00|     NONE|     SURG|
|  57|    58|        31|   5845|04-27-2010 01:30:20|     SURG|     SURG|
|  72|    73|        39|   5853|05-23-2022 15:05:34|    PSYCH|     SURG|
|  79|    80|        43|   5857|12-07-2009 08:31:28|       NB|     SURG|
|  87|    88|        46|   5860|03-06-1996 00:00:00|     NONE|     SURG|
| 105|   106|        56|   5870|06-30-1969 12:41:03|       NB|     SURG|
| 114|   115|        63|   5877|04-10-1987 00:00:00|     NONE|     SURG|
| 130|   131|        70|   5884|06-21-1982 02:10:11|     NMED|     SURG|
| 156|   157|        83|   5897|11-16-2012 00:00:00|     NONE|     SURG|
| 184|   185|       101|   5915|01-19-1993 00:00:00

In [22]:
df5.createOrReplaceTempView("prescriptions_db")
search_5_count = sc.sql("SELECT count(*) as tot_count from prescriptions_db where drug = 'Bisacodyl' ")
# search_5_count.show()

In [23]:
search_5 = sc.sql("SELECT * from prescriptions_db where drug = 'Bisacodyl' ")
search_5.show(137)

+-----+----------+-------+----------+-----------------+---------+---------+-----------------+----------------+-----------+------------+--------------+
|  _c0|subject_id|hadm_id|icustay_id|prescription_date|drug_type|     drug|formulary_drug_cd|   prod_strength|dose_val_rx|dose_unit_rx|form_unit_disp|
+-----+----------+-------+----------+-----------------+---------+---------+-----------------+----------------+-----------+------------+--------------+
|   84|        18|   5832|      9434|       2049-10-17|     MAIN|Bisacodyl|            BISA5|        5 mg Tab|         10|          mg|           TAB|
|   94|        20|   5834|      9436|       2010-10-12|     MAIN|Bisacodyl|            BISA5|        5 mg Tab|         10|          mg|           TAB|
|  180|        37|   5851|      9453|       2012-03-20|     MAIN|Bisacodyl|            BISA5|        5 mg Tab|         10|          mg|           TAB|
|  467|        93|   5907|      9509|       1999-01-18|     MAIN|Bisacodyl|            BISA5| 

In [24]:
search_1.write.mode('overwrite').parquet("gs://dsci-551/Output")
search_2.write.mode('overwrite').parquet("gs://dsci-551/Output")
search_3.write.mode('overwrite').parquet("gs://dsci-551/Output")
search_4.write.mode('overwrite').parquet("gs://dsci-551/Output")
search_5.write.mode('overwrite').parquet("gs://dsci-551/Output")

In [28]:
analytics_1 = sc.sql("""SELECT count(*) as Count from admissions_db a join patients_db p 
                     on a.subject_id = p.subject_id where p.gender = 'M' and a.marital_status = 'SINGLE' 
                     and a.insurance = 'Medicare'""")
analytics_1.show()

+-----+
|Count|
+-----+
|  220|
+-----+



In [25]:
analytics_2 = sc.sql("""SELECT SUM(CASE WHEN year(intime)<2000 then 1 else 0 end) as Admits_before_Year_2000, 
                     SUM(CASE WHEN year(intime)>=2000 then 1 else 0 end) as Admits_after_Year_2000 from icustays_db""")
analytics_2.show()

+-----------------------+----------------------+
|Admits_before_Year_2000|Admits_after_Year_2000|
+-----------------------+----------------------+
|                   1256|                  1244|
+-----------------------+----------------------+



In [26]:
analytics_3 = sc.sql("""(SELECT b.gender,a.drug,count(*) as prescription_count from prescriptions_db a 
join patients_db b on a.subject_id = b.subject_id where b.gender = 'M' group by a.drug,b.gender 
order by prescription_count desc limit 1) UNION 
(SELECT b.gender,a.drug,count(*) as prescription_count from prescriptions_db a join patients_db b 
on a.subject_id = b.subject_id where b.gender = 'F' group by a.drug,b.gender order by prescription_count desc limit 1)""") 
analytics_3.show()

+------+------------------+------------------+
|gender|              drug|prescription_count|
+------+------------------+------------------+
|     M|Potassium Chloride|               335|
|     F|Potassium Chloride|               307|
+------+------------------+------------------+



In [29]:
analytics_1.write.mode('overwrite').parquet("gs://dsci-551/Output")
analytics_2.write.mode('overwrite').parquet("gs://dsci-551/Output")
analytics_3.write.mode('overwrite').parquet("gs://dsci-551/Output")